In [1]:
import sys
import os
import pandas as pd
import numpy as np 
import json
from pathlib import Path

# --- 1. Cấu hình đường dẫn để import từ 'src' ---
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

if str(src_dir) not in sys.path:
    sys.path.append(str(src_dir))

print(f"Đã thêm vào sys.path: {src_dir}")

# --- 2. Import hàm từ file src/data_cleaning.py ---
try:
    # Đảm bảo bạn đã khởi động lại Kernel nếu vừa sửa file .py
    from data_cleaning import preprocess_and_filter_data
    print("Import hàm 'preprocess_and_filter_data' từ 'src' thành công!")
except ImportError as e:
    print(f"LỖI: Không thể import hàm. {e}")
    print("Vui lòng Khởi động lại Kernel (Kernel > Restart Kernel...)")

# --- 3. Cấu hình đường dẫn Data và Config ---
DATA_DIR = project_root / 'data'

# Đường dẫn đến file config CŨ (2019-2024)
CONFIG_PATH_CU = DATA_DIR / 'to_hop_cu.json' 

# Đường dẫn đến file config MỚI (2025)
CONFIG_PATH_MOI = DATA_DIR / 'to_hop_moi.json'

print(f"Config Cũ: {CONFIG_PATH_CU.name}")
print(f"Config Mới: {CONFIG_PATH_MOI.name}")

Đã thêm vào sys.path: d:\Documents University\Semester 5\Intro DS\Project\introduction-to-data-science-feature-crawl-diem-thi\src
Import hàm 'preprocess_and_filter_data' từ 'src' thành công!
Config Cũ: to_hop_cu.json
Config Mới: to_hop_moi.json


In [2]:
# 1. Định nghĩa các năm và file config tương ứng
years_to_process = {
    2019: CONFIG_PATH_CU,
    2020: CONFIG_PATH_CU,
    2021: CONFIG_PATH_CU,
    2022: CONFIG_PATH_CU,
    2023: CONFIG_PATH_CU,
    2024: CONFIG_PATH_CU,
    2025: CONFIG_PATH_MOI,
}

print(f"Bắt đầu xử lý {len(years_to_process)} năm...")
print("-" * 40)

all_stats_list = []
# ===================================================

# 2. Vòng lặp (loop) qua từng năm
for year, config_file in years_to_process.items():
    
    print(f"ĐANG XỬ LÝ NĂM: {year}")
    
    # 2.1. Xác định file input
    if year == 2025:
        input_name = f"diem_thi_toan_quoc_{year}.csv"
    else:
        # File được tạo bởi 'convert_historical_data.py'
        # Giả sử file output của convert_historical_data.py là "diem_thi_toan_quoc_{year}.csv"
        input_name = f"diem_thi_toan_quoc_{year}.csv" 
        
    input_path = DATA_DIR / input_name
    
    # 2.2. Xác định file output
    output_name = f"diem_thi_{year}_new.csv"
    output_path = DATA_DIR / output_name

    try:
        # 2.3. Đọc file
        print(f"Đang đọc: {input_name}")
        df_raw = pd.read_csv(input_path, low_memory=False)
        
        # 2.4. Gọi hàm xử lý
        # Hàm này được import từ src/data_cleaning.py
        df_filtered, year_stats = preprocess_and_filter_data(df_raw, config_file)
        
        year_stats['Năm'] = year
        all_stats_list.append(year_stats)
        # ======================================
        
        # 2.5. Lưu kết quả
        if df_filtered is not None and not df_filtered.empty:
            df_filtered.to_csv(output_path, index=False, encoding='utf-8-sig')
            print(f"✅ ĐÃ LƯU: {output_name} (Số lượng: {len(df_filtered)})")
        else:
            print(f"⚠️ Cảnh báo: Không có dữ liệu nào còn lại cho năm {year} sau khi lọc.")
            
    except FileNotFoundError:
        print(f"LỖI: Không tìm thấy file input '{input_name}'. Bỏ qua năm {year}.")
        # Thêm stats rỗng nếu file lỗi
        all_stats_list.append({'Năm': year, 'Lỗi': f'Không tìm thấy file {input_name}'})
    except Exception as e:
        print(f"LỖI: Một lỗi không xác định đã xảy ra với năm {year}: {e}")
        all_stats_list.append({'Năm': year, 'Lỗi': str(e)})
        
    print("-" * 40)

print("--- TẤT CẢ QUY TRÌNH HOÀN TẤT ---")

# === BƯỚC 3: HIỂN THỊ BẢNG THỐNG KÊ ===
print("\nBẢNG TỔNG HỢP KẾT QUẢ LỌC DỮ LIỆU")

if all_stats_list:
    # 1. Chuyển list các dicts thành DataFrame
    df_stats = pd.DataFrame(all_stats_list)
    
    # 2. Sắp xếp các cột cho dễ đọc
    columns_order = [
        'Năm', 
        'Số lượng trước khi lọc', 
        'Số lượng dữ liệu lỗi (SBD rỗng/trùng lặp)', 
        'Số lượng thí sinh bị điểm liệt', 
        'Số lượng thí sinh tổ hợp không đạt yêu cầu (>=15)', 
        'Số lượng sau khi lọc',
        'Phần % loại bỏ',
        'Lỗi' # Cột này sẽ là NaN nếu không có lỗi
    ]
    
    # Lọc ra các cột thực sự tồn tại trong df_stats
    final_stat_cols = [col for col in columns_order if col in df_stats.columns]
    df_stats = df_stats[final_stat_cols]
    
    # 3. Định dạng lại cột %
    if 'Phần % loại bỏ' in df_stats.columns:
        df_stats['Phần % loại bỏ'] = df_stats['Phần % loại bỏ'].map('{:,.2f}%'.format, na_action='ignore')

    # 4. Hiển thị bảng (dùng display() để Jupyter hiển thị đẹp)
    display(df_stats.set_index('Năm'))
else:
    print("Không có số liệu thống kê nào được thu thập.")

Bắt đầu xử lý 7 năm...
----------------------------------------
ĐANG XỬ LÝ NĂM: 2019
Đang đọc: diem_thi_toan_quoc_2019.csv
✅ ĐÃ LƯU: diem_thi_2019_new.csv (Số lượng: 770477)
----------------------------------------
ĐANG XỬ LÝ NĂM: 2020
Đang đọc: diem_thi_toan_quoc_2020.csv
✅ ĐÃ LƯU: diem_thi_2020_new.csv (Số lượng: 828705)
----------------------------------------
ĐANG XỬ LÝ NĂM: 2021
Đang đọc: diem_thi_toan_quoc_2021.csv
✅ ĐÃ LƯU: diem_thi_2021_new.csv (Số lượng: 943199)
----------------------------------------
ĐANG XỬ LÝ NĂM: 2022
Đang đọc: diem_thi_toan_quoc_2022.csv
✅ ĐÃ LƯU: diem_thi_2022_new.csv (Số lượng: 956811)
----------------------------------------
ĐANG XỬ LÝ NĂM: 2023
Đang đọc: diem_thi_toan_quoc_2023.csv
✅ ĐÃ LƯU: diem_thi_2023_new.csv (Số lượng: 977727)
----------------------------------------
ĐANG XỬ LÝ NĂM: 2024
Đang đọc: diem_thi_toan_quoc_2024.csv
✅ ĐÃ LƯU: diem_thi_2024_new.csv (Số lượng: 1038631)
----------------------------------------
ĐANG XỬ LÝ NĂM: 2025
Đang đọc

,Số lượng trước khi lọc,Số lượng dữ liệu lỗi (SBD rỗng/trùng lặp),Số lượng thí sinh bị điểm liệt,Số lượng thí sinh tổ hợp không đạt yêu cầu (>=15),Số lượng sau khi lọc,Phần % loại bỏ
Năm,,,,,,
2019,882594,0,3671,108446,770477,12.70%
2020,870517,0,1276,40536,828705,4.80%
2021,992295,4591,1246,43259,943199,4.95%
2022,995441,0,1045,37585,956811,3.88%
2023,1022060,0,621,43712,977727,4.34%
2024,1061605,0,462,22512,1038631,2.16%
2025,1131136,0,929,150412,979795,13.38%
